# Match Kitti Sequence version 22-11

Reads a sequence of stereo images as Frame. The Frames are scanned to identify FramePoints that are pixels that are at the top or botrom of vertical lines in the image. A patch is taken around each of the FramePoints in the left hand image, to match these to a patch in the right hand image and estimate the disparity/distance of the framePoint.

A Sequence is created in which the first Frame is the KeyFrame. It's initial FramePoints are culled to keep only the points for which there is a confident estimate of its depth. The consecutive frames are used to track the KeyFrame's remaining FramePoints. Only a limited number of all framePoints will be found in the consecutive frames, mostly because nearby objects become larger quickly as we get closer. The matching FramePoints are given the same FramePointId as the corresponding point in the KeyFrame and its depth is estimated.

The results are dumped into a folder 'dump', containing files 0.txt, 1.txt, etc. 0.txt contains the FramePoints as observed from the first frame, 1.txt form the second frame, etc. In these files, each row contains a single FramePoint with the values FramePointID, X, Y, Depth.

In [1]:
# when you change a setting you have to restart the kernel (settings are only loaded once)
import os
os.environ['STEREO_CONFIDENCE'] = '1.6'
os.environ['SEQUENCE_CONFIDENCE'] = '1.6'
#os.environ['PATCH_SIZE'] = '17'
FRAMECOUNT = 1101

from pyurb.urb_kitti import *
%matplotlib inline
import numpy as np

Read the first 10 frames from KITTI sequence 00.

In [2]:
%%time
LEFTDIR = '/data/urbinn/datasets/kitti/sequences/01/image_2'
RIGHTDIR = '/data/urbinn/datasets/kitti/sequences/01/image_3'

frame = []
for frameid in range(FRAMECOUNT):
    filename = '%06d.png'%(frameid)
    left_frame = Frame(LEFTDIR + '/' + filename, RIGHTDIR)
    frame.append(left_frame)

CPU times: user 389 µs, sys: 928 µs, total: 1.32 ms
Wall time: 1.32 ms


In [ ]:
%%bash
ls /data/urbinn/datasets/kitti/sequences/01/image_2 | wc -l


1101


Inspect the FramePoints for the first frame

We can also manually compute the Depth of the points and show these

Turn the list of frames into a sequence. The first frame is automatically taken as the KeyFrame and its FramePoints are rerieved, depth is estimated, only the most confident points are kept, and they are tracked in the other frames.

In [ ]:
seq = Sequence()
for f in ProgressBar()(frame):
    seq.add_frame(f)

  0% (5 of 1101) |                        | Elapsed Time: 0:00:06 ETA:  0:24:54

invalid speed 4.20373948795 -1.03510464781 
 [[ 0.97477079  0.0036529  -0.22317834  1.76321253]
 [-0.00251859  0.99998243  0.00536694 -0.1015843 ]
 [ 0.22319402 -0.00466944  0.97476286 -0.01759371]
 [ 0.          0.          0.          1.        ]]


  1% (16 of 1101) |                       | Elapsed Time: 0:00:18 ETA:  0:11:38

150 -1.00338851616 [[  9.98943459e-01   9.72309543e-05  -4.59560289e-02  -2.08941232e-02]
 [ -1.12544650e-05   9.99998249e-01   1.87109748e-03   7.17438652e-03]
 [  4.59561304e-02  -1.86860338e-03   9.98941711e-01  -1.00338852e+00]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00   1.00000000e+00]]
invalid speed -2.08640786322 -1.09959396519 
 [[  9.95641204e-01   3.08054441e-03  -9.32153570e-02   6.39715736e-02]
 [ -2.89525640e-03   9.99993555e-01   2.12291185e-03   3.22341005e-02]
 [  9.32212960e-02  -1.84377615e-03   9.95643707e-01  -2.08640786e+00]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00   1.00000000e+00]]


  1% (17 of 1101) |                       | Elapsed Time: 0:00:21 ETA:  0:14:50

...

326 -2.32780922731 [[  9.99992376e-01   1.84617226e-04  -3.90045783e-03   2.17797070e-02]
 [ -1.80446797e-04   9.99999412e-01   1.06954016e-03   1.77352600e-02]
 [  3.90065299e-03  -1.06882818e-03   9.99991821e-01  -2.32780923e+00]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00   1.00000000e+00]]
invalid speed -4.61479654215 0.333528752086 
 [[  9.99975710e-01  -3.42976925e-04  -6.96147193e-03   1.05217210e-02]
 [  3.59894058e-04   9.99996985e-01   2.42900145e-03   3.62954197e-02]
 [  6.96061785e-03  -2.43144784e-03   9.99972819e-01  -4.61479654e+00]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00   1.00000000e+00]]


 10% (118 of 1101) |##                    | Elapsed Time: 0:12:16 ETA:  2:53:31

297 -2.32792425009 [[  9.99991957e-01   1.43807489e-04  -4.00807125e-03   3.08473403e-02]
 [ -1.36577660e-04   9.99998363e-01   1.80403269e-03   1.64381143e-02]
 [  4.00832413e-03  -1.80347077e-03   9.99990340e-01  -2.32792425e+00]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00   1.00000000e+00]]
invalid speed -4.27524502036 -0.553622248639 
 [[  9.99967719e-01   2.30379694e-04  -8.03166850e-03  -5.91669126e-02]
 [ -2.15767221e-04   9.99998320e-01   1.82017606e-03   7.44334985e-02]
 [  8.03207434e-03  -1.81838433e-03   9.99966089e-01  -4.27524502e+00]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00   1.00000000e+00]]


 10% (119 of 1101) |##                    | Elapsed Time: 0:12:30 ETA:  2:58:56

85 -2.32678526262 [[  9.99995547e-01   1.05274271e-03  -2.79259081e-03  -1.12592131e-02]
 [ -1.04920864e-03   9.99998647e-01   1.26667772e-03   3.91781525e-03]
 [  2.79392052e-03  -1.26374207e-03   9.99995298e-01  -2.32678526e+00]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00   1.00000000e+00]]
invalid speed -4.68549423217 -1.14467658663 
 [[  9.99974665e-01  -3.02405616e-04  -7.11177986e-03   2.79025147e-02]
 [  3.17483193e-04   9.99997704e-01   2.11905147e-03  -2.17690682e-03]
 [  7.11112272e-03  -2.12125566e-03   9.99972466e-01  -4.68549423e+00]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00   1.00000000e+00]]


In [ ]:
show2(draw_frame_depth(frame[0]), draw_frame_depth(frame[1]))

Show only the ID's in frame[0] that are actually in frame[1], to avoid clutter in the image.

In [ ]:
#kp0 = [kp for kp in frame[0].get_observations() if kp.get_mappoint() in { p.get_mappoint() for p in frame[1].get_observations() }]
#kp1 = [kp for kp in frame[1].get_observations() if kp.has_mappoint()]
#frame[1].compute_depth()
#show2(draw_observations_depth(kp0), draw_observations_depth(kp1))

In [ ]:
frame[5].compute_depth()
frame[5].filter_not_useful()
frame[6].compute_depth()
frame[6].filter_not_useful()
obs1 = [o for o in frame[5].get_observations() ]
obs2 = [o for o in frame[6].get_observations() ]

In [ ]:
show2(draw_observations_depth(obs1), draw_observations_depth(obs2))

In [ ]:
frame[5].compute_depth()
a = [ (o.get_depth(), o.confidence) for o in frame[5].get_observations() if o.confidence is not None and o.confidence > 1.4 ]
len(a)
#show2(draw_frame(frame[5]), draw_frame(frame[5].get_right_frame()))

In [ ]:
pl = [ o for o in frame[5].get_observations() if o.confidence is not None and o.confidence > 1.4 ]
show(draw_observations_depth(pl))

In [ ]:
show2(draw_frame(frame[0]), draw_frame(f.get_right_frame()))

In [ ]:
f = frame[0]

In [ ]:
f.compute_depth()

In [ ]:
show(draw_observations_depth(frame[0].get_observations()[:100]))